In [ ]:
'''
Steps:
    1) Divide imput image into patches
    2) Flatten the patches to be handles as tokens
    3) Add classification token at the beginning
    4) Feed tokens into the encoder
    5) Number of output of the encoder is equal to the size of the input
    6) 
'''

In [5]:
import torch
import torch.nn as nn 
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
import numpy as np
from torchsummary import summary

In [6]:
# TODO: LOAD OUR DATA 256X256

In [11]:
# https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html

class VisionTransformerCoinImageClassifier(nn.Module):
    def __init__(self, 
                 d_model, 
                 nhead, 
                 num_encoder_layers, 
                 num_decoder_layers, 
                 dim_feedforward, 
                 activation, 
                 norm_first, 
                 patch_size, 
                 image_size, 
                 num_classes
                ):
        super(VisionTransformerCoinImageClassifier, self).__init__()

        # Calculate the number of patches
        num_patches = (image_size // patch_size) ** 2 # **2 = ^2 due to X(W) and Y(H) position considered

        # Path embedding layer - Sliding window - Non-overlapping image patches
        self.patch_embedding = nn.Conv2d(3, d_model, kernel_size=patch_size, stride=patch_size)

        # Positional Encoding - Fixed positional encoding
        self._generate_positional_encoding()
        
        # Encoder - Multihead Attention, FFN and Layer Normalization 
        self._build_encoder()

        # Decoder first layer - Masked Multihead
        self._build_decoder_first_layer()

        # Decoder - Multihead Attention, FFN and Layer Normalization
        self._build_encoder()
    
        # Classification - Linear and Softmax

    def _generate_positional_encoding(self):
        print('Hello PE')

    def _build_encoder(self):
        print('Hello encoder')
        
    
    def _build_decoder_first_layer(self):
        print('Hello decoder first layer')
    
    def _build_decoder(self):
        print('Hello decoder')
    

d_model = 65536
nhead = 8
num_encoder_layers = 6
num_decoder_layers = 6 
num_classes = 2 
norm_first = False 
patch_size = 16 
image_size = 256 # Square it for true size 
activation = 'relu'
dim_feedforward = 2048

model = VisionTransformerCoinImageClassifier(d_model=d_model, 
                                             nhead=nhead, 
                                             num_decoder_layers=num_encoder_layers, 
                                             num_encoder_layers=num_decoder_layers, 
                                             dim_feedforward=dim_feedforward, 
                                             activation=activation, 
                                             norm_first=norm_first,
                                             patch_size=patch_size,
                                             image_size=image_size,
                                             num_classes=num_classes
                                            )

Hello PE
Hello encoder
Hello decoder first layer
Hello encoder
